# Experiment 1: Neural Network

In [1]:
import sys
sys.path.append("../src")

In [2]:
import data.preparation_eurythmy_data as ped
from features.features_dataset import FeaturesDataset
from models.fully_connected_classifier import FullyConnectedClassifier
from evaluation.hyperparameter_tuner import HyperparameterTuner
from collections import Counter

## Import Dataset

In [3]:
feat_dataset_path= r"..\data\processed\features_dataset"
feat_dataset = FeaturesDataset.load(file_path= feat_dataset_path)

In [4]:
feat_dataset.features.shape

(148682, 52)

In [5]:
feat_dataset.features.head()

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy,initial_second,eurythmy_letter,mfcc_1_avg,mfcc_2_avg,...,flatness_ratio_100,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,interquartile_range,skewness,kurtosis,dfa
0,1,1,2023-04-29,salad,1,1,0.0,None,-232.006348,87.030777,...,1.0000,0.000128,8171.888932,1.013423,0.013814,0.117533,0.181950,0.224347,-0.509566,1.586800
1,1,1,2023-04-29,salad,1,1,1.0,None,-250.255188,85.806961,...,0.9894,0.000129,8190.134755,0.865816,0.015855,0.125916,0.181950,0.418608,-0.598494,1.466508
2,1,1,2023-04-29,salad,1,1,2.0,None,-278.646332,68.209419,...,0.9941,0.000077,13453.068166,1.289050,0.054860,0.234221,0.454875,-0.314866,-1.620937,1.336079
3,1,1,2023-04-29,salad,1,1,3.0,None,-276.146942,74.985809,...,1.0000,0.000112,9082.708501,1.374193,0.010778,0.103816,0.090975,-0.524587,0.561958,1.404778
4,1,1,2023-04-29,salad,1,1,4.0,None,-299.724091,62.226551,...,1.0000,0.000085,12209.774692,1.289022,0.029832,0.172718,0.272925,-0.852706,-0.735823,1.509514


## Prepare Data

In [6]:
# Drop constant signals
indexes_constant_value = feat_dataset.features[feat_dataset.features['flatness_ratio_100'] == 1].index.tolist()
feat_dataset.drop_rows(indexes_constant_value)

# Drop columns
columns=['duration_seconds', 'flatness_ratio_10000','flatness_ratio_5000', 'flatness_ratio_1000', 'flatness_ratio_500','flatness_ratio_100']
feat_dataset.drop_columns(columns_to_drop=columns)

In [7]:
feat_dataset.features.shape

(120478, 46)

In [8]:
# Get Train and Validation Indexes
train_indexes, val_indexes, _= ped.get_train_val_test_indexes(df= feat_dataset.features)

# Split the training data
train_feat_dataset= feat_dataset.copy()
train_feat_dataset.features= feat_dataset.features.iloc[train_indexes]
train_feat_dataset.features.reset_index(drop=True, inplace=True)

# Split the validation data
val_feat_dataset= feat_dataset.copy()
val_feat_dataset.features= feat_dataset.features.iloc[val_indexes]
val_feat_dataset.features.reset_index(drop=True, inplace=True)

In [9]:
# Free memory
del feat_dataset

## RQ1

Is there any difference in the signals when someone is performing eurythmy?

### Prepare data

In [10]:
# Reduce the Dataset for the datapoints concerning RQ1

rq1_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(1, rq1_train_feat_dataset.features)
rq1_train_feat_dataset.features= rq1_train_feat_dataset.features.iloc[processed_train_indexes]
rq1_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq1_val_feat_dataset= val_feat_dataset.copy()
processed_val_indexes, val_targets= ped.get_indexes_and_targets_by_rq(1, rq1_val_feat_dataset.features)
rq1_val_feat_dataset.features= rq1_val_feat_dataset.features.iloc[processed_val_indexes]
rq1_val_feat_dataset.features.reset_index(drop=True, inplace=True)

In [11]:
# Reduce the features that are correlated in the training data
train_cols= rq1_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq1_val_feat_dataset.keep_only_specified_variable_columns(train_cols)

Reduced variable features from 38 to 13.


In [12]:
# Normalize features
normalization_params= rq1_train_feat_dataset.normalize_features()
rq1_val_feat_dataset.apply_normalization(normalization_params)

Variable features were properly normalized using 'zscore' method.
Applied z-score normalization.


In [13]:
rq1_train_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_1_std,mfcc_2_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,standard_deviation,skewness,kurtosis,dfa
0,0.162320,1.301609,-0.359987,-0.64417,1.569847,-0.421593,-0.320561,-0.253442,2.022671,-0.571741,0.226188,-0.051912,-1.574697
1,0.291925,1.066949,-0.583099,-0.64417,1.207764,-0.358748,-0.331738,-0.242328,1.745434,-0.564730,-0.556192,-0.033156,-0.705431
2,0.409087,0.995133,-0.535808,-0.64417,1.426345,-0.344245,-0.494149,-0.124977,1.912831,-0.569910,0.206100,-0.021532,-0.378510
3,0.247368,1.258264,0.296119,-0.64417,1.595419,-0.368416,-0.415835,-0.187193,2.042350,-0.578172,-1.250188,0.032878,-0.498403
4,0.499139,0.956201,-0.561084,-0.64417,1.210882,-0.295902,-0.329293,-0.241093,1.737831,-0.272468,-0.198061,-0.056737,-0.379016


In [14]:
rq1_val_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_1_std,mfcc_2_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,standard_deviation,skewness,kurtosis,dfa
0,0.627872,0.216392,0.376850,-0.644170,0.848690,-0.445764,-0.280306,-0.254594,1.455164,-0.214586,-1.007803,0.006051,-1.005731
1,-0.053672,0.761159,0.788414,0.813161,-0.631456,-0.431261,1.902191,-0.781730,0.222839,-0.152319,0.828278,-0.013239,-0.884197
2,0.795006,-0.143439,-0.696042,-0.644170,-0.264342,-0.378085,-0.457318,0.199153,0.550437,-0.066241,1.065417,-0.005489,-0.687476
3,1.112026,-0.005121,-0.728510,-0.644170,0.522713,-0.392587,-0.139713,-0.306833,1.140272,0.332169,-0.268426,-0.067424,-0.459514
4,0.687864,0.238210,-0.172826,-0.644170,0.760842,-0.416759,-0.185556,-0.300125,1.385400,-0.188854,-0.727373,-0.004232,-0.806984


In [15]:
train_loader= rq1_train_feat_dataset.get_variable_features_loader(train_targets)
val_loader= rq1_val_feat_dataset.get_variable_features_loader(val_targets)

### Search

In [19]:
def print_counts_and_percentages(values):
    count = Counter(values)
    total = sum(count.values())
    
    print("Counts and Percentages:")
    for key, value in count.items():
        percentage = (value / total) * 100
        print(f"Class {key}: Count = {value}, Percentage = {percentage:.2f}%")

In [17]:
print_counts_and_percentages(train_targets)

Counts and Percentages:
Class 1: Count = 25000, Percentage = 48.14%
Class 0: Count = 26929, Percentage = 51.86%


In [18]:
%%time
input_size= len(rq1_train_feat_dataset.variable_columns)
output_size= 2
num_epochs = 50

param_grid = {
    'learning_rate': [0.001,0.0003],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0,0.2]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

Training model with parameters: {'learning_rate': 0.001, 'dense_units': 64, 'dense_layers': 1, 'dropout_rate': 0}
Epoch 1/50, Training Loss: 0.6191, Training Accuracy: 0.6624, Validation Loss: 0.6036
Epoch 2/50, Training Loss: 0.6073, Training Accuracy: 0.6741, Validation Loss: 0.6068
Epoch 3/50, Training Loss: 0.6030, Training Accuracy: 0.6774, Validation Loss: 0.6065
Epoch 4/50, Training Loss: 0.6001, Training Accuracy: 0.6782, Validation Loss: 0.6014
Epoch 5/50, Training Loss: 0.5986, Training Accuracy: 0.6815, Validation Loss: 0.6019
Epoch 6/50, Training Loss: 0.5961, Training Accuracy: 0.6818, Validation Loss: 0.5973
Epoch 7/50, Training Loss: 0.5953, Training Accuracy: 0.6817, Validation Loss: 0.6016
Epoch 8/50, Training Loss: 0.5938, Training Accuracy: 0.6831, Validation Loss: 0.6039
Epoch 9/50, Training Loss: 0.5921, Training Accuracy: 0.6820, Validation Loss: 0.6160
Epoch 10/50, Training Loss: 0.5918, Training Accuracy: 0.6831, Validation Loss: 0.6140
Epoch 11/50, Training Los

In [19]:
print("Best Hyperparameters: \n", best_params)

Best Hyperparameters: 
 learning_rate        0.001000
dense_units        128.000000
dense_layers         1.000000
dropout_rate         0.200000
validation_loss      0.594889
Name: 5, dtype: float64


In [20]:
all_results.head(30)

,learning_rate,dense_units,dense_layers,dropout_rate,validation_loss
0,0.0010,64,1,0.0,0.606729
1,0.0010,64,1,0.2,0.601381
2,0.0010,64,2,0.0,0.611023
3,0.0010,64,2,0.2,0.629416
4,0.0010,128,1,0.0,0.626335
5,0.0010,128,1,0.2,0.594889
6,0.0010,128,2,0.0,0.617622
7,0.0010,128,2,0.2,0.621385
8,0.0003,64,1,0.0,0.604256
9,0.0003,64,1,0.2,0.602602


In [21]:
del rq1_train_feat_dataset, rq1_val_feat_dataset

## RQ2

Is there any difference in the signals between different eurythmy letters?

### Prepare data

In [55]:
# Reduce the Dataset for the datapoints concerning RQ2

rq2_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(2, rq2_train_feat_dataset.features)
rq2_train_feat_dataset.features= rq2_train_feat_dataset.features.iloc[processed_train_indexes]
rq2_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq2_val_feat_dataset= val_feat_dataset.copy()
processed_val_indexes, val_targets= ped.get_indexes_and_targets_by_rq(2, rq2_val_feat_dataset.features)
rq2_val_feat_dataset.features= rq2_val_feat_dataset.features.iloc[processed_val_indexes]
rq2_val_feat_dataset.features.reset_index(drop=True, inplace=True)

In [56]:
# Reduce the features that are correlated in the training data
train_cols= rq2_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq2_val_feat_dataset.keep_only_specified_variable_columns(train_cols)

Reduced variable features from 38 to 18.


In [57]:
# Normalize features
normalization_params= rq2_train_feat_dataset.normalize_features()
rq2_val_feat_dataset.apply_normalization(normalization_params)

Variable features were properly normalized using 'zscore' method.
Applied z-score normalization.


In [58]:
rq2_train_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_3_avg,mfcc_9_avg,mfcc_1_std,mfcc_3_std,mfcc_10_std,mfcc_13_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,interquartile_range,skewness,kurtosis,dfa
0,0.181529,2.132812,1.598015,1.138220,0.323217,-0.057176,-0.053062,-0.5526,1.563628,-0.317001,-0.324180,-0.237665,1.927491,-0.122706,-0.450211,0.145849,-0.052391,-1.332307
1,0.317012,1.053640,1.362902,0.906860,-0.168501,-0.194491,-0.205548,-0.5526,1.199946,-0.235926,-0.334743,-0.225651,1.658217,-0.122379,-0.450211,-0.557272,-0.036517,-0.418069
2,0.439488,0.981281,1.447960,0.836054,-0.208711,-0.183580,-0.075705,-0.5526,1.419492,-0.217217,-0.488247,-0.098795,1.820806,-0.122621,-0.591050,0.127796,-0.026679,-0.074234
3,0.270435,1.547587,1.520059,1.095484,0.366835,0.042590,0.029634,-0.5526,1.589313,-0.248399,-0.414229,-0.166050,1.946605,-0.122997,-0.591050,-1.180964,0.019369,-0.200330
4,0.533625,0.769814,0.809706,0.797669,-0.236783,0.146524,-0.039744,-0.5526,1.203077,-0.154852,-0.332432,-0.224316,1.650832,-0.100332,-0.027692,-0.235421,-0.056474,-0.074766


In [59]:
rq2_val_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_3_avg,mfcc_9_avg,mfcc_1_std,mfcc_3_std,mfcc_10_std,mfcc_13_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,interquartile_range,skewness,kurtosis,dfa
0,0.668196,0.527876,1.345406,0.068265,-0.835092,-0.539043,-0.592788,-0.552600,0.839286,-0.348183,-0.286132,-0.238911,1.376284,-0.094011,-0.199487,-0.963133,-0.003335,-0.733905
1,-0.044258,0.113357,-1.023485,0.605371,0.391572,-0.406861,-0.340529,1.091183,-0.647398,-0.329474,1.776664,-0.808740,0.179357,-0.086490,-0.199487,0.686946,-0.019661,-0.606083
2,0.842910,0.228102,-0.167630,-0.286506,-0.506078,-0.740345,-0.812728,-0.552600,-0.278663,-0.260872,-0.453435,0.251587,0.497545,-0.074861,-0.022019,0.900061,-0.013102,-0.399185
3,1.174309,0.333494,0.194877,-0.150132,-0.856613,-0.431178,-0.582200,-0.552600,0.511869,-0.279582,-0.153250,-0.295380,1.070438,-0.002423,0.687852,-0.298658,-0.065519,-0.159428
4,0.730909,0.687299,0.789595,0.089776,-0.114606,-0.404026,-0.418825,-0.552600,0.751049,-0.310764,-0.196579,-0.288129,1.308524,-0.090994,-0.199487,-0.711112,-0.012038,-0.524876


In [60]:
train_loader= rq2_train_feat_dataset.get_variable_features_loader(train_targets)
val_loader= rq2_val_feat_dataset.get_variable_features_loader(val_targets)

### Parameters Search

In [61]:
print_counts_and_percentages(train_targets)

Counts and Percentages:
Class 0: Count = 7824, Percentage = 35.11%
Class 1: Count = 6977, Percentage = 31.31%
Class 2: Count = 7481, Percentage = 33.57%


In [62]:
%%time
input_size= len(rq2_train_feat_dataset.variable_columns)
output_size= 3
num_epochs = 50

param_grid = {
    'learning_rate': [0.001,0.0003],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0,0.2]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

Training model with parameters: {'learning_rate': 0.001, 'dense_units': 64, 'dense_layers': 1, 'dropout_rate': 0}
Epoch 1/50, Training Loss: 1.1011, Training Accuracy: 0.3438, Validation Loss: 1.0977
Epoch 2/50, Training Loss: 1.0968, Training Accuracy: 0.3556, Validation Loss: 1.0997
Epoch 3/50, Training Loss: 1.0952, Training Accuracy: 0.3630, Validation Loss: 1.0991
Epoch 4/50, Training Loss: 1.0942, Training Accuracy: 0.3671, Validation Loss: 1.0980
Epoch 5/50, Training Loss: 1.0936, Training Accuracy: 0.3657, Validation Loss: 1.0996
Epoch 6/50, Training Loss: 1.0932, Training Accuracy: 0.3655, Validation Loss: 1.0977
Early stopping triggered
Early stopping triggered after 5 epochs
Training model with parameters: {'learning_rate': 0.001, 'dense_units': 64, 'dense_layers': 1, 'dropout_rate': 0.2}
Epoch 1/50, Training Loss: 1.1036, Training Accuracy: 0.3500, Validation Loss: 1.0971
Epoch 2/50, Training Loss: 1.0983, Training Accuracy: 0.3584, Validation Loss: 1.0966
Epoch 3/50, Train

In [63]:
print("Best Hyperparameters: \n", best_params)

Best Hyperparameters: 
 learning_rate        0.000300
dense_units        128.000000
dense_layers         2.000000
dropout_rate         0.200000
validation_loss      1.096048
Name: 15, dtype: float64


In [64]:
all_results.head(30)

,learning_rate,dense_units,dense_layers,dropout_rate,validation_loss
0,0.0010,64,1,0.0,1.097720
1,0.0010,64,1,0.2,1.098068
2,0.0010,64,2,0.0,1.097688
3,0.0010,64,2,0.2,1.097586
4,0.0010,128,1,0.0,1.103352
5,0.0010,128,1,0.2,1.097339
6,0.0010,128,2,0.0,1.098991
7,0.0010,128,2,0.2,1.097370
8,0.0003,64,1,0.0,1.097204
9,0.0003,64,1,0.2,1.096756


In [65]:
del rq2_train_feat_dataset, rq2_val_feat_dataset

## RQ3

Is there any eurythmy habituation in the plant between different days?

### Prepare data

In [66]:
# Reduce the Dataset for the datapoints concerning RQ3

rq3_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(3, rq3_train_feat_dataset.features)
rq3_train_feat_dataset.features= rq3_train_feat_dataset.features.iloc[processed_train_indexes]
rq3_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq3_val_feat_dataset= val_feat_dataset.copy()
processed_val_indexes, val_targets= ped.get_indexes_and_targets_by_rq(3, rq3_val_feat_dataset.features)
rq3_val_feat_dataset.features= rq3_val_feat_dataset.features.iloc[processed_val_indexes]
rq3_val_feat_dataset.features.reset_index(drop=True, inplace=True)

In [67]:
# Reduce the features that are correlated in the training data
train_cols= rq3_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq3_val_feat_dataset.keep_only_specified_variable_columns(train_cols)

Reduced variable features from 38 to 14.


In [68]:
# Normalize features
normalization_params= rq3_train_feat_dataset.normalize_features()
rq3_val_feat_dataset.apply_normalization(normalization_params)

Variable features were properly normalized using 'zscore' method.
Applied z-score normalization.


In [69]:
rq3_train_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_13_avg,mfcc_7_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,skewness,kurtosis,dfa
0,0.182807,1.412749,-0.193645,-0.552407,1.566174,-0.315942,-0.321771,-0.238539,1.954898,-0.121210,-0.533098,0.142784,-0.051145,-1.320496
1,0.318564,1.125377,-0.320592,-0.552407,1.201535,-0.237575,-0.332322,-0.226428,1.685691,-0.120886,-0.525305,-0.545511,-0.036273,-0.414100
2,0.441286,1.152323,-0.177436,-0.552407,1.421659,-0.219490,-0.485659,-0.098549,1.848240,-0.121126,-0.531063,0.125112,-0.027056,-0.073215
3,0.271892,1.341358,-0.003890,-0.552407,1.591927,-0.249631,-0.411721,-0.166346,1.974007,-0.121498,-0.540247,-1.156051,0.016085,-0.198230
4,0.535612,0.683289,-0.195795,-0.552407,1.204674,-0.159208,-0.330014,-0.225082,1.678309,-0.099027,-0.200451,-0.230447,-0.054970,-0.073743


In [70]:
rq3_val_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_13_avg,mfcc_7_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,skewness,kurtosis,dfa
0,0.670454,0.826452,-0.250549,-0.552407,0.839925,-0.346083,-0.283764,-0.239795,1.403830,-0.092760,-0.136115,-0.942814,-0.005186,-0.727227
1,-0.043435,-0.821615,0.350241,1.083224,-0.650673,-0.327999,1.776788,-0.814219,0.207203,-0.085302,-0.066904,0.672471,-0.020481,-0.600502
2,0.845521,-0.406876,-0.741062,-0.552407,-0.280967,-0.261688,-0.450886,0.254659,0.525311,-0.073773,0.028773,0.881093,-0.014336,-0.395378
3,1.177586,-0.063770,-0.627145,-0.552407,0.511646,-0.279773,-0.151027,-0.296719,1.098060,-0.001952,0.471612,-0.292350,-0.063444,-0.157679
4,0.733294,0.521204,-0.406807,-0.552407,0.751456,-0.309914,-0.194309,-0.289410,1.336087,-0.089768,-0.107513,-0.696107,-0.013340,-0.519991


In [71]:
train_loader= rq3_train_feat_dataset.get_variable_features_loader(train_targets)
val_loader= rq3_val_feat_dataset.get_variable_features_loader(val_targets)

### Parameters search

In [72]:
print_counts_and_percentages(train_targets)

Counts and Percentages:
Class 0: Count = 16137, Percentage = 64.55%
Class 1: Count = 3251, Percentage = 13.00%
Class 2: Count = 3494, Percentage = 13.98%
Class 3: Count = 2118, Percentage = 8.47%


In [73]:
%%time
input_size= len(rq3_train_feat_dataset.variable_columns)
output_size= 4
num_epochs = 50

param_grid = {
    'learning_rate': [0.001,0.0003],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0,0.2]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

Training model with parameters: {'learning_rate': 0.001, 'dense_units': 64, 'dense_layers': 1, 'dropout_rate': 0}
Epoch 1/50, Training Loss: 1.0231, Training Accuracy: 0.6438, Validation Loss: 1.2927
Epoch 2/50, Training Loss: 0.9881, Training Accuracy: 0.6511, Validation Loss: 1.2908
Epoch 3/50, Training Loss: 0.9786, Training Accuracy: 0.6534, Validation Loss: 1.2901
Epoch 4/50, Training Loss: 0.9738, Training Accuracy: 0.6550, Validation Loss: 1.3091
Epoch 5/50, Training Loss: 0.9687, Training Accuracy: 0.6567, Validation Loss: 1.2834
Epoch 6/50, Training Loss: 0.9652, Training Accuracy: 0.6562, Validation Loss: 1.3239
Epoch 7/50, Training Loss: 0.9622, Training Accuracy: 0.6587, Validation Loss: 1.3206
Epoch 8/50, Training Loss: 0.9587, Training Accuracy: 0.6590, Validation Loss: 1.2828
Epoch 9/50, Training Loss: 0.9568, Training Accuracy: 0.6594, Validation Loss: 1.3126
Epoch 10/50, Training Loss: 0.9540, Training Accuracy: 0.6600, Validation Loss: 1.3305
Epoch 11/50, Training Los

In [74]:
print("Best Hyperparameters: \n", best_params)

Best Hyperparameters: 
 learning_rate       0.000300
dense_units        64.000000
dense_layers        1.000000
dropout_rate        0.000000
validation_loss     1.281168
Name: 8, dtype: float64


In [75]:
all_results.head(30)

,learning_rate,dense_units,dense_layers,dropout_rate,validation_loss
0,0.0010,64,1,0.0,1.299304
1,0.0010,64,1,0.2,1.304015
2,0.0010,64,2,0.0,1.371862
3,0.0010,64,2,0.2,1.341775
4,0.0010,128,1,0.0,1.341619
5,0.0010,128,1,0.2,1.286907
6,0.0010,128,2,0.0,1.350864
7,0.0010,128,2,0.2,1.321973
8,0.0003,64,1,0.0,1.281168
9,0.0003,64,1,0.2,1.293265


In [76]:
del rq3_train_feat_dataset, rq3_val_feat_dataset

## RQ4

Is there any eurythmy habituation in the plant between several gestures in the same performance?

### Prepare data

In [10]:
# Reduce the Dataset for the datapoints concerning RQ4

rq4_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(4, rq4_train_feat_dataset.features)
rq4_train_feat_dataset.features= rq4_train_feat_dataset.features.iloc[processed_train_indexes]
rq4_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq4_val_feat_dataset= val_feat_dataset.copy()
processed_val_indexes, val_targets= ped.get_indexes_and_targets_by_rq(4, rq4_val_feat_dataset.features)
rq4_val_feat_dataset.features= rq4_val_feat_dataset.features.iloc[processed_val_indexes]
rq4_val_feat_dataset.features.reset_index(drop=True, inplace=True)

In [12]:
# Reduce the features that are correlated in the training data
train_cols= rq4_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq4_val_feat_dataset.keep_only_specified_variable_columns(train_cols)

Reduced variable features from 38 to 17.


In [13]:
# Normalize features
normalization_params= rq4_train_feat_dataset.normalize_features()
rq4_val_feat_dataset.apply_normalization(normalization_params)


Variable features were properly normalized using 'zscore' method.
Applied z-score normalization.


In [14]:
rq4_train_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_8_avg,mfcc_13_avg,mfcc_1_std,mfcc_3_std,mfcc_12_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,skewness,kurtosis,dfa
0,0.182807,1.720903,1.412749,1.131024,0.316808,-0.092646,-0.552407,1.566174,-0.315942,-0.321771,-0.238539,1.954898,-0.121210,-0.533098,0.142784,-0.051145,-1.320496
1,0.318564,1.391997,1.125377,0.900143,-0.173381,-0.212038,-0.552407,1.201535,-0.237575,-0.332322,-0.226428,1.685691,-0.120886,-0.525305,-0.545511,-0.036273,-0.414100
2,0.441286,1.414190,1.152323,0.829483,-0.213466,-0.079056,-0.552407,1.421659,-0.219490,-0.485659,-0.098549,1.848240,-0.121126,-0.531063,0.125112,-0.027056,-0.073215
3,0.271892,1.492432,1.341358,1.088377,0.360291,0.010282,-0.552407,1.591927,-0.249631,-0.411721,-0.166346,1.974007,-0.121498,-0.540247,-1.156051,0.016085,-0.198230
4,0.535612,0.794197,0.683289,0.791179,-0.241450,-0.083798,-0.552407,1.204674,-0.159208,-0.330014,-0.225082,1.678309,-0.099027,-0.200451,-0.230447,-0.054970,-0.073743


In [15]:
rq4_val_feat_dataset.objective_features.head()

,mfcc_1_avg,mfcc_8_avg,mfcc_13_avg,mfcc_1_std,mfcc_3_std,mfcc_12_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,skewness,kurtosis,dfa
0,0.670454,1.257000,0.826452,0.063284,-0.837900,-0.006426,-0.552407,0.839925,-0.346083,-0.283764,-0.239795,1.403830,-0.092760,-0.136115,-0.942814,-0.005186,-0.727227
1,-0.043435,-1.176580,-0.821615,0.599278,0.384951,0.152043,1.083224,-0.650673,-0.327999,1.776788,-0.814219,0.207203,-0.085302,-0.066904,0.672471,-0.020481,-0.600502
2,0.845521,-0.074428,-0.406876,-0.290752,-0.509909,-0.897396,-0.552407,-0.280967,-0.261688,-0.450886,0.254659,0.525311,-0.073773,0.028773,0.881093,-0.014336,-0.395378
3,1.177586,0.344850,-0.063770,-0.154661,-0.859354,-0.560735,-0.552407,0.511646,-0.279773,-0.151027,-0.296719,1.098060,-0.001952,0.471612,-0.292350,-0.063444,-0.157679
4,0.733294,0.888370,0.521204,0.084751,-0.119654,-0.382283,-0.552407,0.751456,-0.309914,-0.194309,-0.289410,1.336087,-0.089768,-0.107513,-0.696107,-0.013340,-0.519991


In [16]:
train_loader= rq4_train_feat_dataset.get_variable_features_loader(train_targets)
val_loader= rq4_val_feat_dataset.get_variable_features_loader(val_targets)

### Search

In [20]:
print_counts_and_percentages(train_targets)

Counts and Percentages:
Class 0: Count = 6707, Percentage = 26.83%
Class 1: Count = 6284, Percentage = 25.14%
Class 2: Count = 6224, Percentage = 24.90%
Class 3: Count = 5785, Percentage = 23.14%


In [18]:
%%time
input_size= len(rq4_train_feat_dataset.variable_columns)
output_size= 4
num_epochs = 50

param_grid = {
    'learning_rate': [0.001,0.0003],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0,0.2]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

Training model with parameters: {'learning_rate': 0.001, 'dense_units': 64, 'dense_layers': 1, 'dropout_rate': 0}
Epoch 1/50, Training Loss: 1.3794, Training Accuracy: 0.2828, Validation Loss: 1.3661
Epoch 2/50, Training Loss: 1.3724, Training Accuracy: 0.2966, Validation Loss: 1.3668
Epoch 3/50, Training Loss: 1.3706, Training Accuracy: 0.3005, Validation Loss: 1.3689
Epoch 4/50, Training Loss: 1.3691, Training Accuracy: 0.3026, Validation Loss: 1.3654
Epoch 5/50, Training Loss: 1.3676, Training Accuracy: 0.3069, Validation Loss: 1.3682
Epoch 6/50, Training Loss: 1.3663, Training Accuracy: 0.3073, Validation Loss: 1.3652
Epoch 7/50, Training Loss: 1.3655, Training Accuracy: 0.3116, Validation Loss: 1.3646
Epoch 8/50, Training Loss: 1.3647, Training Accuracy: 0.3079, Validation Loss: 1.3683
Epoch 9/50, Training Loss: 1.3636, Training Accuracy: 0.3130, Validation Loss: 1.3646
Epoch 10/50, Training Loss: 1.3633, Training Accuracy: 0.3148, Validation Loss: 1.3672
Epoch 11/50, Training Los

In [21]:
print("Best Hyperparameters: \n", best_params)

Best Hyperparameters: 
 learning_rate       0.000300
dense_units        64.000000
dense_layers        2.000000
dropout_rate        0.200000
validation_loss     1.363252
Name: 11, dtype: float64


In [22]:
all_results.head(30)

,learning_rate,dense_units,dense_layers,dropout_rate,validation_loss
0,0.0010,64,1,0.0,1.370320
1,0.0010,64,1,0.2,1.364896
2,0.0010,64,2,0.0,1.372483
3,0.0010,64,2,0.2,1.366218
4,0.0010,128,1,0.0,1.370432
5,0.0010,128,1,0.2,1.367533
6,0.0010,128,2,0.0,1.370149
7,0.0010,128,2,0.2,1.368428
8,0.0003,64,1,0.0,1.364877
9,0.0003,64,1,0.2,1.365013
